# This notebook creates a consolidated and curated 5-year world happiness data from year 2020 to 2024.

### Create a binary parquet file for world map

In [1]:
import geopandas as gpd

# Define the URL of the shapefile
url = "https://naciscdn.org/naturalearth/110m/cultural/ne_110m_admin_0_countries.zip"

# Read the shapefile from the URL using GeoPandas
geo_countries = (gpd.read_file(url)
                 [[ "NAME", "CONTINENT", "geometry"]]
                 .rename(columns={"NAME": "Country", 
                                  "CONTINENT": "Continent", 
                                  "geometry": "geometry"}))

# Save the GeoDataFrame as a Parquet file & csv file
geo_countries.to_parquet('../data/processed/world_countries.parquet')
geo_countries.to_csv('../data/processed/world_countries.csv')

# Print success message
print("Parquet file saved as 'world_countries.parquet'")
print("Parquet file saved as 'world_countries.csv'")

Parquet file saved as 'world_countries.parquet'
Parquet file saved as 'world_countries.csv'


In [2]:
# Read the Parquet file into a GeoDataFrame
geo_countries_from_parquet = gpd.read_parquet('../data/processed/world_countries.parquet')

# Print the first few rows to confirm the data
print(geo_countries_from_parquet.head())

                    Country      Continent  \
0                      Fiji        Oceania   
1                  Tanzania         Africa   
2                 W. Sahara         Africa   
3                    Canada  North America   
4  United States of America  North America   

                                            geometry  
0  MULTIPOLYGON (((180 -16.06713, 180 -16.55522, ...  
1  POLYGON ((33.90371 -0.95, 34.07262 -1.05982, 3...  
2  POLYGON ((-8.66559 27.65643, -8.66512 27.58948...  
3  MULTIPOLYGON (((-122.84 49, -122.97421 49.0025...  
4  MULTIPOLYGON (((-122.84 49, -120 49, -117.0312...  


### Create a cleaned and consolidated 5-year world happiness data (2020-2024)

In [3]:
import requests
import pandas as pd
from io import BytesIO
import pycountry_convert as pc
from sklearn.preprocessing import MinMaxScaler

# ---------------------------
# Sourcing: Load Raw Data from URLs
# ---------------------------
files_to_download = [
    ("https://happiness-report.s3.amazonaws.com/2020/WHR20_DataForFigure2.1.xls", 2020),
    ("https://happiness-report.s3.amazonaws.com/2021/DataForFigure2.1WHR2021C2.xls", 2021),
    ("https://happiness-report.s3.amazonaws.com/2022/Appendix_2_Data_for_Figure_2.1.xls", 2022),
    ("https://happiness-report.s3.amazonaws.com/2023/DataForFigure2.1WHR2023.xls", 2023),
    ("https://happiness-report.s3.amazonaws.com/2024/DataForFigure2.1+with+sub+bars+2024.xls", 2024),
]

def fetch_excel_data(url, engine="xlrd"):
    """Fetch an Excel file from a URL and return a DataFrame."""
    response = requests.get(url)
    response.raise_for_status()  # Raise error if response is not successful
    return pd.read_excel(BytesIO(response.content), engine=engine)

raw_data = {}
for url, year in files_to_download:
    # Use "xlrd" for .xls files; for .xlsx, use "openpyxl"
    engine = "xlrd" if url.endswith("xls") else "openpyxl"
    df = fetch_excel_data(url, engine=engine)
    df["Year"] = year
    # Drop rows missing the happiness score (column might be named "Ladder score" or "Happiness score")
    if "Ladder score" in df.columns:
        df = df.dropna(subset=["Ladder score"])
    elif "Happiness score" in df.columns:
        df = df.dropna(subset=["Happiness score"])
    raw_data[year] = df

# ---------------------------
# Staging: Clean and Standardize DataFrames
# ---------------------------
COLUMN_MAPPING = {
    'Country name': 'Country',
    'Country': 'Country',
    'Ladder score': 'Happiness Score',  # Standardize to 'Happiness Score'
    'Happiness score': 'Happiness Score',
    'Logged GDP per capita': 'GDP per Capita',
    'Explained by: Log GDP per capita': 'GDP per Capita',
    'Explained by: GDP per capita': 'GDP per Capita',
    'Social support': 'Social Support',
    'Explained by: Social support': 'Social Support',
    'Healthy life expectancy': 'Healthy Life Expectancy',
    'Explained by: Healthy life expectancy': 'Healthy Life Expectancy',
    'Freedom to make life choices': 'Freedom to Make Life Choices',
    'Explained by: Freedom to make life choices': 'Freedom to Make Life Choices',
    'Generosity': 'Generosity',
    'Explained by: Generosity': 'Generosity',
    'Perceptions of corruption': 'Perceptions of Corruption',
    'Explained by: Perceptions of corruption': 'Perceptions of Corruption',
    'Year': 'Year'
}

REQUIRED_COLUMNS = [
    'Year', 'Country', 'Happiness Score', 'GDP per Capita', 
    'Social Support', 'Healthy Life Expectancy', 'Freedom to Make Life Choices', 
    'Generosity', 'Perceptions of Corruption'
]

def clean_dataframe(df):
    """Strip whitespace and trailing '*' characters from string columns."""
    for col in df.select_dtypes(include=["object"]).columns:
        df[col] = df[col].str.strip().str.rstrip('*')
    return df

# Initialize MinMaxScaler
scaler = MinMaxScaler()

staged_data = {}
for year, df in raw_data.items():
    # Rename columns based on mapping
    df = df.rename(columns=COLUMN_MAPPING)
    # Remove duplicate columns if any
    df = df.loc[:, ~df.columns.duplicated()]
    # Clean data
    df = clean_dataframe(df)
    # Retain only the required columns (if available)
    cols_to_keep = [col for col in REQUIRED_COLUMNS if col in df.columns]
    df = df[cols_to_keep].dropna(subset=cols_to_keep)
    # Normalize the numeric columns using MinMaxScaler (except 'Country' and 'Year')
    numeric_columns = [col for col in df.columns if col not in ['Country', 'Year']]   
    # Apply scaling
    df[numeric_columns] = scaler.fit_transform(df[numeric_columns])    
    # Store the processed dataframe
    staged_data[year] = df

# ---------------------------
# Integration: Combine Datasets and Add Continent Mapping
# ---------------------------
# Combine all staged DataFrames into one.
merged_df = pd.concat(staged_data.values(), ignore_index=True)

# Correct specific country names for consistency.
merged_df["Country"] = merged_df["Country"].replace({
    "Taiwan Province of China": "Taiwan",
    "Eswatini, Kingdom of": "Eswatini",
    "Turkiye": "Turkey",
    # "Congo": "Congo (Brazzaville)",
    "Bosnia and Herzegovina": "Bosnia and Herz.", 
    "Central African Republic": "Central African Rep.",
    "Congo (Brazzaville)": "Congo",
    "Congo (Kinshasa)": "Dem. Rep. Congo",
    "Czech Republic": "Czechia",
    "Dominican Republic": "Dominican Rep.",
    "Hong Kong S.A.R. of China": "China",
    "Ivory Coast": "Côte d'Ivoire",
    "Macedonia": "North Macedonia",
    "United States": "United States of America",
    "Eswatini": "eSwatini",
    "Swaziland": "eSwatini",
    "South Sudan": "S. Sudan",
})

# Use regex to replace anything that starts with 'Congo' in the 'Country' column
merged_df["Country"] = merged_df["Country"].str.replace(r'^Brazzaville.*', 'Congo', regex=True)

# Reorder columns for readability.
final_columns = [
    "Year", "Country", "Happiness Score", "GDP per Capita", 
    "Social Support", "Healthy Life Expectancy", "Freedom to Make Life Choices", 
    "Generosity", "Perceptions of Corruption"
]
merged_df = merged_df.sort_values(by=["Year", "Country"])[final_columns]

# Group by 'Country' and 'Year' and calculate the mean of the relevant columns.
grouped_df = merged_df.groupby(['Country', 'Year'], as_index=False)[
    ["Happiness Score", "GDP per Capita", "Social Support", 
     "Healthy Life Expectancy", "Freedom to Make Life Choices", 
     "Generosity", "Perceptions of Corruption"]].mean()

# Now, merged_df contains the mean values for each country across the years.
merged_df = grouped_df

# Define a function to map a country name to its continent using pycountry_convert.
def get_continent(country_name):
    try:
        country_alpha2 = pc.country_name_to_country_alpha2(country_name)
        continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
        continent_name = pc.convert_continent_code_to_continent_name(continent_code)
        return continent_name
    except Exception:
        return "Unknown"

# Apply the continent mapping to add a new "Continent" column.
merged_df["Continent"] = merged_df["Country"].apply(get_continent)

# Apply manual continent updates to merged_df
manual_continent_updates = {
    # "Congo (Brazzaville)": "Africa",
    # "Congo (Kinshasa)": "Africa",
    "Hong Kong S.A.R. of China": "Asia",
    "Kosovo": "Europe",
    "North Cyprus": "Asia",
    "Palestinian Territories": "Asia",
    "Dem. Rep. Congo": "Africa",
    "S. Sudan": "Africa",
    "Bosnia and Herz.": "Europe",
    "Central African Rep.": "Africa",
    "Dominican Rep.": "North America",
    "eSwatini": "Africa"
}

for country, continent in manual_continent_updates.items():
    merged_df.loc[merged_df["Country"] == country, "Continent"] = continent

# Reorder columns to include the 'Continent' column.
final_columns = [
    "Year", "Country", "Continent", "Happiness Score", "GDP per Capita", 
    "Social Support", "Healthy Life Expectancy", "Freedom to Make Life Choices", 
    "Generosity", "Perceptions of Corruption"
]
merged_df = merged_df[final_columns]

print("✅ Integration step completed. The final merged dataset is stored in 'merged_df'.")
print(merged_df.head())

# ---------------------------
# Analysis: Calculate Average Continent KPIs
# ---------------------------
# Ensure KPI columns are numeric.
numeric_cols = [
    "Happiness Score", "GDP per Capita", "Social Support", "Healthy Life Expectancy", 
    "Freedom to Make Life Choices", "Generosity", "Perceptions of Corruption"
]
merged_df[numeric_cols] = merged_df[numeric_cols].apply(pd.to_numeric, errors="coerce")

# Calculate average KPIs by Year and Continent.
regional_avg = merged_df.groupby(["Year", "Continent"])[numeric_cols].mean().reset_index()

# Rename KPI columns for clarity.
regional_avg = regional_avg.rename(columns={
    "Happiness Score": "Average Continent Happiness Score",
    "GDP per Capita": "Average Continent GDP per Capita",
    "Social Support": "Average Continent Social Support",
    "Healthy Life Expectancy": "Average Continent Healthy Life Expectancy", 
    "Freedom to Make Life Choices": "Average Continent Freedom to Make Life Choices", 
    "Generosity": "Average Continent Generosity", 
    "Perceptions of Corruption": "Average Continent Perceptions of Corruption"
})

# Merge the calculated averages back into the main dataset.
merged_df = merged_df.merge(regional_avg, on=["Year", "Continent"], how="left")

print("✅ ETL pipeline completed. The final dataset is stored in 'merged_df'.")
print(merged_df.head())

# ---------------------------
# Reporting: Save Final DataFrame to CSV
# ---------------------------
output_csv = "../data/processed/World_Happiness_processed_data.csv"
merged_df.to_csv(output_csv, index=False)
print(f"✅ Final processed data saved to '{output_csv}'.")

output_parquet = "../data/processed/World_Happiness_processed_data.parquet"
merged_df.to_parquet(output_parquet, index=False)
print(f"✅ Final processed data saved to '{output_parquet}'.")

✅ Integration step completed. The final merged dataset is stored in 'merged_df'.
   Year      Country Continent  Happiness Score  GDP per Capita  \
0  2020  Afghanistan      Asia              0.0        0.195686   
1  2021  Afghanistan      Asia              0.0        0.211402   
2  2022  Afghanistan      Asia              0.0        0.343077   
3  2023  Afghanistan      Asia              0.0        0.293045   
4  2024  Afghanistan      Asia              0.0        0.293451   

   Social Support  Healthy Life Expectancy  Freedom to Make Life Choices  \
0        0.230319                 0.233827                           0.0   
1        0.000000                 0.140988                           0.0   
2        0.000000                 0.306453                           0.0   
3        0.000000                 0.123583                           0.0   
4        0.000000                 0.282000                           0.0   

   Generosity  Perceptions of Corruption  
0    0.237331   